## <b> 데이터 시각화 </b>
#### 인지실험 데이터 시각화 관련 프로그램

In [269]:
import os
import pandas as pd
import numpy as np
import random
import math
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
import scipy
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import FVA.FVA as fva
from FVA.lpc import *
from FVA.detect import *
from FVA.mfcc import *
from FVA.animation import *
from FVAmanager import *
from DB.DBmanager import *
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
#-*-coding:utf-8-*-
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

## <b> Data Load </b>

In [270]:
nF1, nF2        =   10, 10
result, users   =   getDB()
formants        =   pd.read_excel("DB/sounds.xlsx")
vowels          =   {'ㅏ':0,'ㅓ':1,'ㅗ':2,'ㅜ':3,'ㅡ':4,'ㅣ':5,'ㅐ':6,'ㅔ':7}
colors          =   ['red','green','orange','dodgerblue','deeppink','aqua','blueviolet','yellow']
counts          =   np.zeros((8,nF1,nF2))
userDict        =   {a[0]:a[1] for a in users.values.tolist()}

In [271]:
def toymd(x):
   return x.split('T')[0]

def countSetup(namefilter=None,timefilter=None):
   """
   Args:
      namefilter (list): [names]
      timefilter (list): [start, end] with "2023-10-03" style
   """
   counts = np.zeros((8,nF1,nF2))
   for x in result.values.tolist():
      if (namefilter is None) or (userDict.get(x[4],None) in namefilter):
         if (timefilter is None) or (timefilter[0]<=toymd(x[3]) and timefilter[0]<=toymd(x[3])):
            counts[vowels[x[2]]][int(x[1].split('_')[0])-1][int(x[1].split('_')[1].split('.')[0])-1] += 1
   
   return counts.transpose(1,2,0)

In [272]:
NameFilter = ['노수빈']
TimeFilter = None#['2023-09-27','2023-10-03']

In [273]:
board = countSetup(namefilter=NameFilter,timefilter=TimeFilter)
print(board.sum())

1225.0


## <b> Data Visuallization

In [191]:
k = 800

def softmax(x): # numpy array!!
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def probability(a): # numpy array!!
    sum = a.sum()
    return a/sum

def print_randDot(fig,xrange:list,yrange:list,color,prob:float):
    """
    Args:
        fig (figure): 
        xrange (list): [xmin, xmax]
        yrange (list): [ymin, ymax]
        color (str): color of dot (plt.scatter(color) style)
        prob (float): probability of dot in [0,1]
    """
    nDot = int(round(prob * k, 0))
    x = [random.uniform(yrange[0],yrange[1]) for i in range(nDot)]
    y = [random.uniform(xrange[0],xrange[1]) for i in range(nDot)]
    plt.scatter(x,y,c=color,marker='o',s=6,alpha=0.25)
    
fig = plt.figure()
ax  = fig.add_subplot()
plt.gca().set_aspect('equal', adjustable='box')
ax.set_ylim([0,nF1])
ax.set_xlim([0,nF2])
for i in range(nF1):
    for q in range(nF2):
        a = probability(np.array(board[i][q]))
        for w in range(8):
            print_randDot(fig,[9-i,10-i],[9-q,10-q],colors[w],a[w])

for i,vowel in enumerate(vowels.keys()):
    plt.scatter([],[],s=50,c=colors[i],marker='o',label=vowel)
plt.rc('legend', fontsize=20)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))

plt.show()

In [274]:
fig, ax = plt.subplots(nF1,nF2+1)
plt.gca().set_aspect('equal', adjustable='box')
for i in range(nF1):
    for q in range(nF2+1):
        ax[i][q].xaxis.set_visible(False)
        ax[i][q].yaxis.set_visible(False)
        ax[i][q].axis('off')
        if q<nF2:
            ax[i][nF2-1-q].pie(np.array(board[i][q]),colors=colors,radius=1.3)
for i,vowel in enumerate(vowels.keys()):
    ax[nF1-1][nF2].scatter([],[],s=150,c=colors[i],marker='o',label=vowel)

plt.rc('legend', fontsize=20)
plt.legend(loc='lower left', bbox_to_anchor=(0.0,5.0))
plt.show()